In [1]:
import sys
sys.path.append('..')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import torch
import pandas as pd
import seaborn as sns
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from experiment_setup import build_estimator

from model.cnn import SimpleConv, Trainer

In [15]:
config = {
    'use_cuda': True,
    'batch_size': 128,
    'epochs': 10,
    'lr': 1e-2,
    'momentum': 0.5,
    'seed': 1,
    'log_interval': 10
}



In [4]:
x, y = fetch_openml('mnist_784', return_X_y=True, cache=True)

In [5]:
%%time
x_ood, _ = fetch_openml('Fashion-MNIST', return_X_y=True, cache=True)

In [6]:
x = x.reshape(-1, 1, 28, 28)
x_ood = x_ood.reshape(-1, 1, 28, 28)
y = y.astype('long').reshape(-1)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=10_000, stratify=y)
x_train /= 255.0
x_val /= 255.0


In [16]:
train_samples = 3000
model = SimpleConv()
trainer = Trainer(model)
trainer.fit(x_train[:train_samples], y_train[:train_samples], epochs=config['epochs'])
accuracy_score(y_val, trainer.predict(x_val))

In [17]:
estimator = build_estimator('bald', trainer, num_classes=10)

In [18]:
estimator.estimate(x_train[:10])

In [19]:
uq_train = estimator.estimate(x_train[:1000])
uq_val = estimator.estimate(x_val[:1000])
uq_ood = estimator.estimate(x_ood[:1000])

In [20]:
df = pd.DataFrame()
df['uq'] = np.concatenate(
    (uq_train, uq_val, uq_ood))
df['dataset'] = ['train'] * len(uq_train) + ['val'] * len(uq_val) + ['ood']*len(uq_ood)

sns.boxplot(data=df, x='dataset',  y='uq')


In [21]:
print('train')
print(pd.Series(uq_train).describe())
print()

print('val')
print(pd.Series(uq_val).describe())
print()

print('ood')
print(pd.Series(uq_ood).describe())
print()
